In [2]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
import nltk
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

In [5]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if t not in NLTK_StopWords]
    return T

In [3]:
text=preprocessing('saya mencoba menggunakan preprocessing')
print (text)

['mencoba', 'preprocessing']


In [4]:
df =pd.read_csv("news.csv")
df.head()

title  \
0  Bikin Atap Bus 'Raib' Usai Tabrak Flyover, Sop...   
1  Mobil Tabrak Pembatas Jalan Tol Jakarta-Tanger...   
2  Tabrak Pembatas Jalan Tol Cisumwadu, Bagian De...   
3    Mobil Masuk Jurang di Ciamis, 2 Orang Tewas\n\n   
4  2 Insiden Kecelakaan Tunggal, Ada Apa di Km 16...   

                                date  \
0  Minggu, 30 Jan 2022 18:41 WIB\n\n   
1  Minggu, 30 Jan 2022 17:35 WIB\n\n   
2  Minggu, 30 Jan 2022 16:27 WIB\n\n   
3  Minggu, 30 Jan 2022 15:43 WIB\n\n   
4  Minggu, 30 Jan 2022 17:21 WIB\n\n   

                                         description  
0  Bikin Atap Bus 'Raib' Usai Tabrak Flyover, Sop...  
1  Mobil Tabrak Pembatas Jalan Tol Jakarta-Tanger...  
2  Tabrak Pembatas Jalan Tol Cisumwadu, Bagian De...  
3  Mobil Masuk Jurang di Ciamis, 2 Orang Tewas\n\...  
4  2 Insiden Kecelakaan Tunggal, Ada Apa di Km 16...

In [17]:
column = ['Title', 'Date', 'Description',]
df_total = pd.DataFrame()
document_text = []
df =pd.read_csv("news.csv")
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
for x in range(0, df.shape[0]):
    text = preprocessing (df.iloc[x, -1])
    document_text.append(df.iloc[x, -1])
    df.iloc[x, -1] = (' '.join(text)) 

In [6]:
teks = []

for i in range(df.shape[0]):
        teks.append(df.iloc[i, -1])

In [7]:
print (teks)

['bikin atap bus raib tabrak flyover sopir diburu polisi sopir bus berkelir merah menabrak kapolres padang akbp novianto taryono sopir bus bernama manalu kabur tabrakan jalan soekarno hatta simpang lapan kota pengendara bus kabur kecelakaan pencarian novianto dikutip peristiwa mengakibatkan atap bus terlepas 17 penumpang mengalami luka luka dirawat rsud padang bus dievakuasi petugas polres kabid humas polda sumbar kombes satake bayu bus melaju arah bukittinggi kota padang sopir bus rute menerobos jalan dilengkapi marka jalan satake dikutip kecepatan bus menghindari tabrakan korban jiwa penumpang mengalami luka luka akibat benturan keras mobil hilang atasnya', 'mobil tabrak pembatas jalan tol jakarta tangerang pengemudi luka mobil kepala induk pjr tol bitung akp suwito membenarkan kejadian tepatnya km 21 600 b arah jakarta kecelakaan minggu 30 1 2022 14 30 wib minibus bernopol a 1281 xw dikendarai abdul hafid bersenggolan mobil dikenal kaget banting setir mengakibatkan menabrak pembatas

In [8]:
print(df.info())
print ('-'*90)
print("Panjang Dokumen Teks : ", len(document_text))
#print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        27 non-null     object
 1   date         27 non-null     object
 2   description  27 non-null     object
dtypes: object(3)
memory usage: 864.0+ bytes
None
------------------------------------------------------------------------------------------
Panjang Dokumen Teks :  27


In [9]:
df.head()

title  \
0  Bikin Atap Bus 'Raib' Usai Tabrak Flyover, Sop...   
1  Mobil Tabrak Pembatas Jalan Tol Jakarta-Tanger...   
2  Tabrak Pembatas Jalan Tol Cisumwadu, Bagian De...   
3    Mobil Masuk Jurang di Ciamis, 2 Orang Tewas\n\n   
4  2 Insiden Kecelakaan Tunggal, Ada Apa di Km 16...   

                                date  \
0  Minggu, 30 Jan 2022 18:41 WIB\n\n   
1  Minggu, 30 Jan 2022 17:35 WIB\n\n   
2  Minggu, 30 Jan 2022 16:27 WIB\n\n   
3  Minggu, 30 Jan 2022 15:43 WIB\n\n   
4  Minggu, 30 Jan 2022 17:21 WIB\n\n   

                                         description  
0  bikin atap bus raib tabrak flyover sopir dibur...  
1  mobil tabrak pembatas jalan tol jakarta tanger...  
2  tabrak pembatas jalan tol cisumwadu mobil rusa...  
3  mobil masuk jurang ciamis 2 orang tewas mobil ...  
4  2 insiden kecelakaan tunggal km 165 166 tol ci...

In [10]:
# Simpan teks asli dokumen
joblib.dump(document_text, "document_text.pkl")
# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('df_total.csv',index=False )


In [11]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(teks)

print('tfidf_matrix')
print(tfidf_matrix.shape)

tfidf_matrix
(27, 1057)


In [12]:
joblib.dump(tfidf_vectorizer, "vectorizer.pkl")
joblib.dump(tfidf_matrix, "tfidf.pkl")

['tfidf.pkl']

### Pencarian 

In [3]:
df_total = pd.read_csv('df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)
document_text= joblib.load('document_text.pkl')
print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        27 non-null     object
 1   date         27 non-null     object
 2   description  27 non-null     object
dtypes: object(3)
memory usage: 864.0+ bytes
None
------------------------------------------------------------------------------------------
27


In [6]:
kueri='lalin truk di jakarta'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "tfidf.pkl" )
tfidf_vectorizer = joblib.load( "vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

['lalin truk jakarta']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  11
Tanggal        :  Sabtu, 29 Jan 2022 20:52 WIB


Isi berita     :  sopir diduga ngantuk 2 truk tabrakan terperosok tol janger truk menabrak truk bermuatan barang berhenti bahu jalan kepala induk pjr tol bitung akp suwito kecelakaan sabtu 29 1 04 45 wib kecelakaan tepatnya km 14 200 a tol jakarta tangerang suwito kronologi kendaraan jakarta serang kendaraan mengecek ban bahu jalan suwito truk mengecek ban truk nomor polisi b 9449 nn berjalan kondisi sopir mengantuk menabrak truk kendaraan berjalan lajur mengantuk menabrak kendaraan kendaraan masuk kejadian truk terperosok selokan samping bahu jalan tol kejadian mengakibatkan kernet truk mengalami luka luka posisi kendaraan bahu jalan kendaraan masuk korban orang
(Score: 0.2990) 
2
No ID Dokumen  :  26
Tanggal        :  Jumat, 28 Jan 2022 10:34 WIB


Isi berita     :  truk muatan kimia tabrak truk kontainer tol tangerang sopir tewas menabrak truk kon